In [1]:
import torch 

In [2]:
device = torch.accelerator.current_accelerator().type

In [3]:
device 

'cuda'

# Transform

In [4]:
# ToTensor converts a PIL image or NumPy ndarray into a FloatTensor. and scales the image's pixel intensity values in the range [0., 1.]
from torchvision.transforms import Lambda
# ---- target_transform definition ----
target_transform = Lambda(
    lambda y: torch.zeros(10, dtype=torch.float)
                    .scatter_(0, torch.tensor(y), value=1)
)

# ---- test it with different labels ----
for y in [0, 3, 5, 9]:
    one_hot = target_transform(y)
    print(f"Label: {y}")
    print(one_hot)
    print()


Label: 0
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Label: 3
tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

Label: 5
tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

Label: 9
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])



In [5]:
z = torch.zeros(10, dtype=torch.float)
z

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [6]:
z.scatter_(0, torch.tensor(5), value=1)

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [7]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [8]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


# Just Forward pass of Data from model 

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [10]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


In [12]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [13]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [14]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [15]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [17]:
from torchinfo import summary

summary(
    model,
    input_size=(1, 28, 28),   # same as your input
    device=device
)


Layer (type:depth-idx)                   Output Shape              Param #
NeuralNetwork                            [1, 10]                   --
├─Flatten: 1-1                           [1, 784]                  --
├─Sequential: 1-2                        [1, 10]                   --
│    └─Linear: 2-1                       [1, 512]                  401,920
│    └─ReLU: 2-2                         [1, 512]                  --
│    └─Linear: 2-3                       [1, 512]                  262,656
│    └─ReLU: 2-4                         [1, 512]                  --
│    └─Linear: 2-5                       [1, 10]                   5,130
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.67
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 2.68
Estimated Total Size (MB): 2.69